# Script transfer learning et réduction de dimension

In [1]:
print("Welcome to my EMR Notebook!")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1687334280043_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Welcome to my EMR Notebook!

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1687334280043_0001,pyspark,idle,Link,Link,None,✔


In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Configuration des paths

In [4]:
# Images + Output + Outpu PCA + Output CSV: 

PATH = 's3://projetfruits'
PATH_Data = PATH+'/data/input/Test'
PATH_Data_sample = PATH+'/data/input/sample'
PATH_Result = PATH+'/data/output'
PATH_Result_pca = PATH+'/data/output_pca'
PATH_Result_csv = PATH+'/data/output_csv'

# Output sample 
PATH_Result_sample = PATH+'/data_sample/output'
PATH_Result_pca_sample = PATH+'/data_sample/output_pca'
PATH_Result_csv_sample = PATH+'/data_sample/output_csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
print('PATH:        '+ PATH + '\nPATH_Data:   '+ PATH_Data + '\nPATH_Result: '+ PATH_Result + '\nPATH_Result_pca: ' + PATH_Result_pca + '\nPATH_Result_CSV: ' + PATH_Result_csv)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://projetfruits
PATH_Data:   s3://projetfruits/data/input/Test
PATH_Result: s3://projetfruits/data/output
PATH_Result_pca: s3://projetfruits/data/output_pca
PATH_Result_CSV: s3://projetfruits/data/output_csv

# Importation des images

Les images sont situées dans un bucket S3, en suivant le chemin : `s3://projetfruits/data/input/Test`

In [6]:
# Pour comparer à l'exécution en local :
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://projetfruits...|2023-06-18 09:35:06|125135|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-18 09:35:06|124785|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-18 09:35:04|123514|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-18 09:35:08|122958|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-18 09:35:05|122807|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

On conserve le chemin d'accès à l'image, et nous ajoutons une colonne labels de chaque image:

`withColumn()`: est une fonction de transformation de DataFrame qui est utilisée pour modifier la valeur, convertir le type de données d'une colonne existante, créer une nouvelle colonne, etc.

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+--------------------------------------------------------+-----------+
|path                                                    |label      |
+--------------------------------------------------------+-----------+
|s3://projetfruits/data/input/Test/apple_hit_1/r0_115.jpg|apple_hit_1|
|s3://projetfruits/data/input/Test/apple_hit_1/r0_119.jpg|apple_hit_1|
|s3://projetfruits/data/input/Test/apple_hit_1/r0_107.jpg|apple_hit_1|
|s3://projetfruits/data/input/Test/apple_hit_1/r0_143.jpg|apple_hit_1|
|s3://projetfruits/data/input/Test/apple_hit_1/r0_111.jpg|apple_hit_1|
+--------------------------------------------------------+-----------+
only showing top 5 rows

None

# MobileNetV2

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 [==============================] - 1s 0us/step

In [10]:
new_model = Model(inputs=model.input, outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Spark Broadcast 

Dans Spark RDD et DataFrame, les variables de diffusion sont des variables partagées en lecture seule qui sont mises en cache et disponibles sur tous les nœuds d'un cluster afin d'accéder ou d'utiliser les tâches.

Au lieu d'envoyer ces données avec chaque tâche, Spark distribue des variables de diffusion à la machine en utilisant des algorithmes de diffusion efficaces pour réduire les coûts de communication.

Au lieu de distribuer ces informations avec chaque tâche sur le réseau (surcharge et perte de temps), nous pouvons utiliser la variable de diffusion pour mettre en cache ces informations de recherche sur chaque machine et les tâches utilisent ces informations mises en cache lors de l'exécution des transformations.

## Comment fonctionne Spark Broadcast ?

Les variables de diffusion sont utilisées de la même manière pour RDD, DataFrame et Dataset. Lorsque vous exécutez un RDD Spark, des jobs DataFrame dont les variables de diffusion sont définies et utilisées, Spark effectue les opérations suivantes:

Spark breaks the job into stages that have distributed shuffling and actions are executed with in the stage.
Later Stages are also broken into tasks
Spark broadcasts the common data (reusable) needed by tasks within each stage.
The broadcasted data is cache in serialized format and deserialized before executing each task.

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt/yarn/usercache/livy/appcache/application_1687334280043_0001/container_1687334280043_0001_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [15]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://projetfruits/data/output

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://projetfruits/data/input/Test/apple_hit_1/...  ...  [0.21089473, 0.08602117, 1.3680781, 0.0, 0.353...
1  s3://projetfruits/data/input/Test/apple_hit_1/...  ...  [0.00890907, 0.20073263, 2.0521815, 0.0, 0.176...
2  s3://projetfruits/data/input/Test/apple_hit_1/...  ...  [0.21464187, 0.24185033, 0.29451635, 0.0, 0.0,...
3  s3://projetfruits/data/input/Test/apple_hit_1/...  ...  [0.0, 0.4196838, 0.25495315, 0.0, 0.3626423, 0...
4  s3://projetfruits/data/input/Test/cabbage_whit...  ...  [0.0, 0.5918541, 1.0169322, 0.0030952473, 1.66...

[5 rows x 3 columns]

In [76]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [77]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3110, 3)

# ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
model_resnet = ResNet50(include_top=False)
model_resnet.summary() 

In [ ]:
bc_model_weights = sc.broadcast(model.get_weights())
def model_fn():
    model_resnet = ResNet50(weights=None, include_top=False)
    model_resnet.set_weights(bc_model_weights.value)
    return model

In [ ]:
features_df_resnet = images.repartition(16).select(col("path"), featurize_udf("content").alias("features"))

# ACP

In [23]:
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.functions import array_to_vector, vector_to_array
from pyspark.ml.feature import StandardScaler, PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_for_pca = spark.read.parquet(PATH_Result)
df_for_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+
|                path|      label|            features|
+--------------------+-----------+--------------------+
|s3://projetfruits...|apple_hit_1|[0.263347, 0.1623...|
|s3://projetfruits...|apple_hit_1|[0.58557224, 1.87...|
|s3://projetfruits...|apple_hit_1|[0.26966664, 0.97...|
|s3://projetfruits...|apple_hit_1|[1.0179651, 0.093...|
|s3://projetfruits...|apple_hit_1|[0.5515656, 0.021...|
|s3://projetfruits...|apple_hit_1|[0.34845087, 0.02...|
+--------------------+-----------+--------------------+
only showing top 6 rows

In [21]:
# conversion du type de la colonne 'features' en vecteur pyspark
df_for_pca = df_for_pca.withColumn('features', array_to_vector('features'))
df_for_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+
|                path|      label|            features|
+--------------------+-----------+--------------------+
|s3://projetfruits...|apple_hit_1|[0.26334699988365...|
|s3://projetfruits...|apple_hit_1|[0.58557224273681...|
|s3://projetfruits...|apple_hit_1|[0.26966664195060...|
|s3://projetfruits...|apple_hit_1|[1.01796507835388...|
|s3://projetfruits...|apple_hit_1|[0.55156558752059...|
|s3://projetfruits...|apple_hit_1|[0.34845086932182...|
+--------------------+-----------+--------------------+
only showing top 6 rows

In [24]:
scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df_for_pca)

# when we transform the dataframe, the old
# feature will still remain in it
df_scaled = scaler.transform(df_for_pca)
df_scaled.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+--------------------+
|                path|      label|            features|      scaledFeatures|
+--------------------+-----------+--------------------+--------------------+
|s3://projetfruits...|apple_hit_1|[0.26334699988365...|[-0.2368718905644...|
|s3://projetfruits...|apple_hit_1|[0.58557224273681...|[0.52002851643245...|
|s3://projetfruits...|apple_hit_1|[0.26966664195060...|[-0.2220271825266...|
|s3://projetfruits...|apple_hit_1|[1.01796507835388...|[1.53571031207079...|
|s3://projetfruits...|apple_hit_1|[0.55156558752059...|[0.44014758920947...|
|s3://projetfruits...|apple_hit_1|[0.34845086932182...|[-0.0369646677646...|
+--------------------+-----------+--------------------+--------------------+
only showing top 6 rows

In [25]:
# PCA
pca = PCA(k=1280, inputCol="scaledFeatures", outputCol="pca_features")
model_pca = pca.fit(df_scaled)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
eigenvalues = model_pca.explainedVariance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(eigenvalues) + 1), eigenvalues, 'bo-')
plt.xlabel("Nombre de composantes")
plt.ylabel("Valeur propre")
plt.title("Éboulis des valeurs propres")
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
No module named 'matplotlib'
Traceback (most recent call last):
ModuleNotFoundError: No module named 'matplotlib'



In [ ]:
cumulative_variance = [sum(eigenvalues[:i + 1]) for i in range(len(eigenvalues))]
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'bo-')
plt.xlabel("Nombre de composantes")
plt.ylabel("Variance expliquée cumulée")
plt.title("Variance expliquée cumulée")
plt.show()

In [28]:
df_pca = model_pca.transform(df_scaled)
df_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+--------------------+--------------------+
|                path|      label|            features|      scaledFeatures|        pca_features|
+--------------------+-----------+--------------------+--------------------+--------------------+
|s3://projetfruits...|apple_hit_1|[0.26334699988365...|[-0.2368718905644...|[7.84086092739329...|
|s3://projetfruits...|apple_hit_1|[0.58557224273681...|[0.52002851643245...|[13.8263061048599...|
|s3://projetfruits...|apple_hit_1|[0.26966664195060...|[-0.2220271825266...|[10.7537572021108...|
|s3://projetfruits...|apple_hit_1|[1.01796507835388...|[1.53571031207079...|[8.97627058951143...|
|s3://projetfruits...|apple_hit_1|[0.55156558752059...|[0.44014758920947...|[0.39619802082023...|
|s3://projetfruits...|apple_hit_1|[0.34845086932182...|[-0.0369646677646...|[-2.7473368106115...|
+--------------------+-----------+--------------------+--------------------+--------------------+
only showing top 6 r

In [29]:
# Variance expliquée
print('Explained Variance Ratio', model_pca.explainedVariance.toArray())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio [1.13364770e-01 6.57112418e-02 4.43377388e-02 ... 3.40246610e-06
 3.36348628e-06 3.15446956e-06]

In [30]:
# Calcul de la variance expliquée cumulée
cumulative_variance = np.cumsum(model_pca.explainedVariance.toArray())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Trouvez l'indice où la variance expliquée cumulée atteint 99%
index = np.argmax(cumulative_variance >= 0.99)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
print('Le nombre de composante retenue : ', index + 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Le nombre de composante retenue :  763

In [33]:
# On refait une ACP avec 763 composantes : 
# PCA
pca = PCA(k=763, inputCol="scaledFeatures", outputCol="pca_features")
model_pca = pca.fit(df_scaled)
df_pca = model_pca.transform(df_scaled)
df_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+--------------------+--------------------+
|                path|      label|            features|      scaledFeatures|        pca_features|
+--------------------+-----------+--------------------+--------------------+--------------------+
|s3://projetfruits...|apple_hit_1|[0.26334699988365...|[-0.2368718905644...|[7.84086092739329...|
|s3://projetfruits...|apple_hit_1|[0.58557224273681...|[0.52002851643245...|[13.8263061048599...|
|s3://projetfruits...|apple_hit_1|[0.26966664195060...|[-0.2220271825266...|[10.7537572021108...|
|s3://projetfruits...|apple_hit_1|[1.01796507835388...|[1.53571031207079...|[8.97627058951143...|
|s3://projetfruits...|apple_hit_1|[0.55156558752059...|[0.44014758920947...|[0.39619802082023...|
|s3://projetfruits...|apple_hit_1|[0.34845086932182...|[-0.0369646677646...|[-2.7473368106115...|
+--------------------+-----------+--------------------+--------------------+--------------------+
only showing top 6 r

In [34]:
# On ne garde pas la colonne 'features' et on convertit 'pca_features' en array
df_pca = df_pca.select(['path', 'label', vector_to_array('pca_features').alias('pca_features')])
df_pca.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+
|                path|      label|        pca_features|
+--------------------+-----------+--------------------+
|s3://projetfruits...|apple_hit_1|[7.84086092739329...|
|s3://projetfruits...|apple_hit_1|[13.8263061048599...|
|s3://projetfruits...|apple_hit_1|[10.7537572021108...|
|s3://projetfruits...|apple_hit_1|[8.97627058951143...|
|s3://projetfruits...|apple_hit_1|[0.39619802082023...|
|s3://projetfruits...|apple_hit_1|[-2.7473368106115...|
+--------------------+-----------+--------------------+
only showing top 6 rows

In [35]:
# Enregistrement des données traitées au format "parquet" :
df_pca.repartition(20).write.mode("overwrite").parquet(PATH_Result_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
# not sure if this is the best way to do it
# X_pca = df_pca.rdd.map(lambda row: row.pca_features).collect()
# X_pca = np.array(X_pca)
# X_pca

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Lecture des résultat pca
df__pca = pd.read_parquet(PATH_Result_pca, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
df__pca.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                       pca_features
0  s3://projetfruits/data/input/Test/apple_granny...  ...  [-8.994687589337751, -4.418026679211101, -9.78...
1  s3://projetfruits/data/input/Test/apple_red_3/...  ...  [-8.832459293805302, 0.9020027901600762, -0.50...
2  s3://projetfruits/data/input/Test/apple_rotten...  ...  [-1.9237638095643004, 5.599823326841137, 13.15...
3  s3://projetfruits/data/input/Test/apple_golden...  ...  [-6.554354766166086, -7.773149964976603, -3.56...
4  s3://projetfruits/data/input/Test/apple_golden...  ...  [-3.578943798445791, -9.845000667619491, 5.172...

[5 rows x 3 columns]

In [41]:
print(spark.sparkContext.getConf().get("spark.driver.host"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ip-172-31-34-247.eu-west-3.compute.internal

In [42]:
configurations = spark.sparkContext.getConf().getAll()
for item in configurations: print(item)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('spark.eventLog.enabled', 'true')
('spark.executor.memory', '9486M')
('spark.driver.extraJavaOptions', "-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -XX:OnOutOfMemoryError='kill -9 %p'")
('spark.sql.parquet.output.committer.class', 'com.amazon.emr.committer.EmrOptimizedSparkSqlParquetOutputCommitter')
('spark.blackli

# Test save au format csv Features

In [48]:
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col,lit
from pyspark.sql.types import StringType, FloatType

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
df = spark.read.parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

In [71]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|            features|
+--------------------+--------------------+--------------------+
|s3://projetfruits...|         apple_hit_1|[0.263347, 0.1623...|
|s3://projetfruits...|         apple_hit_1|[0.58557224, 1.87...|
|s3://projetfruits...|         apple_hit_1|[0.26966664, 0.97...|
|s3://projetfruits...|         apple_hit_1|[1.0179651, 0.093...|
|s3://projetfruits...|         apple_hit_1|[0.5515656, 0.021...|
|s3://projetfruits...|         apple_hit_1|[0.34845087, 0.02...|
|s3://projetfruits...|     cabbage_white_1|[0.0, 0.0971243, ...|
|s3://projetfruits...|         apple_hit_1|[0.2735786, 1.128...|
|s3://projetfruits...|         apple_hit_1|[0.37599656, 0.08...|
|s3://projetfruits...|     cabbage_white_1|[0.0, 0.62696445,...|
|s3://projetfruits...|         apple_hit_1|[0.7772309, 0.062...|
|s3://projetfruits...|         apple_hit_1|[1.212185, 0.0840...|
|s3://projetfruits...|   

In [72]:
# Nombre de features : 
num_features = len(df.select("features").first()[0]) 
num_features

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1280

In [73]:
# Extraction des composante avec udf function
extract_features = udf(lambda x, i: float(x[i]), FloatType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
extract_features

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7fc8f6700d40>

In [75]:
for i in range(num_features):
    df = df.withColumn(f"feature_{i+1}", extract_features("features", lit(i)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- feature_1: float (nullable = true)
 |-- feature_2: float (nullable = true)
 |-- feature_3: float (nullable = true)
 |-- feature_4: float (nullable = true)
 |-- feature_5: float (nullable = true)
 |-- feature_6: float (nullable = true)
 |-- feature_7: float (nullable = true)
 |-- feature_8: float (nullable = true)
 |-- feature_9: float (nullable = true)
 |-- feature_10: float (nullable = true)
 |-- feature_11: float (nullable = true)
 |-- feature_12: float (nullable = true)
 |-- feature_13: float (nullable = true)
 |-- feature_14: float (nullable = true)
 |-- feature_15: float (nullable = true)
 |-- feature_16: float (nullable = true)
 |-- feature_17: float (nullable = true)
 |-- feature_18: float (nullable = true)
 |-- feature_19: float (nullable = true)
 |-- feature_20: float (nullable = true)
 |-- feature_21: float (n

In [78]:
# Colonne à enregistrer : 
selected_columns = ["path", "label"] + [f"feature_{i+1}" for i in range(num_features)]
df_selected = df.select(*selected_columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
df_selected.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- feature_1: float (nullable = true)
 |-- feature_2: float (nullable = true)
 |-- feature_3: float (nullable = true)
 |-- feature_4: float (nullable = true)
 |-- feature_5: float (nullable = true)
 |-- feature_6: float (nullable = true)
 |-- feature_7: float (nullable = true)
 |-- feature_8: float (nullable = true)
 |-- feature_9: float (nullable = true)
 |-- feature_10: float (nullable = true)
 |-- feature_11: float (nullable = true)
 |-- feature_12: float (nullable = true)
 |-- feature_13: float (nullable = true)
 |-- feature_14: float (nullable = true)
 |-- feature_15: float (nullable = true)
 |-- feature_16: float (nullable = true)
 |-- feature_17: float (nullable = true)
 |-- feature_18: float (nullable = true)
 |-- feature_19: float (nullable = true)
 |-- feature_20: float (nullable = true)
 |-- feature_21: float (nullable = true)
 |-- feature_22: float (nullable = true)
 |-- feature_23: float (nulla

In [126]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
# Méthode 01
# output_path = PATH_Result_csv + "/features_csv_1/"
df_selected.write.csv("s3://projetfruits/data/output_csv/features_csv_1/", header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o17149.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)


In [81]:
# Méthode 02
output_path = PATH_Result_csv + "/features_csv_2/"
df_selected.write.mode("overwrite").option("header","true").csv(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o15020.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)


In [131]:
# Méthode 03
output_path = PATH_Result_csv + "/features_csv_3/"
df_selected.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o17155.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)


In [38]:
output_path = PATH_Result_csv + "/features_csv_3"
Test_2 = pd.read_csv(output_path + "/" + ".csv")
Test_2.shape

(451, 1282)

# Test save au format csv PCA

In [43]:

df_pca = spark.read.parquet(PATH_Result_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:

df_pca.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: double (containsNull = true)

In [45]:

df_pca.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+
|                path|               label|        pca_features|
+--------------------+--------------------+--------------------+
|s3://projetfruits...|     zucchini_dark_1|[39.2881919097146...|
|s3://projetfruits...|apple_granny_smith_1|[-6.1524401694463...|
|s3://projetfruits...|         apple_red_1|[-7.1715484957184...|
|s3://projetfruits...|      apple_golden_1|[-2.7984515542920...|
|s3://projetfruits...|   eggplant_violet_1|[33.8626339573818...|
|s3://projetfruits...|      apple_golden_2|[-6.9864907416723...|
|s3://projetfruits...|  apple_red_yellow_1|[-4.6006637263156...|
|s3://projetfruits...|     zucchini_dark_1|[31.2386923656436...|
|s3://projetfruits...|         apple_red_2|[-6.5208107440238...|
|s3://projetfruits...|      apple_rotten_1|[2.66772855622555...|
|s3://projetfruits...|  apple_red_yellow_1|[-4.8280094690883...|
|s3://projetfruits...|          cucumber_1|[9.13805422140912...|
|s3://projetfruits...|app

In [46]:
# Nombre de composante principale : 
num_components = len(df_pca.select("pca_features").first()[0]) 
num_components

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

763

In [49]:
# Extraction des composante avec udf function
extract_component = udf(lambda x, i: float(x[i]), FloatType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
extract_component

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7fc8fd25ca70>

In [51]:
for i in range(num_components):
    df_pca = df_pca.withColumn(f"component_{i+1}", extract_component("pca_features", lit(i)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df_pca.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- component_1: float (nullable = true)
 |-- component_2: float (nullable = true)
 |-- component_3: float (nullable = true)
 |-- component_4: float (nullable = true)
 |-- component_5: float (nullable = true)
 |-- component_6: float (nullable = true)
 |-- component_7: float (nullable = true)
 |-- component_8: float (nullable = true)
 |-- component_9: float (nullable = true)
 |-- component_10: float (nullable = true)
 |-- component_11: float (nullable = true)
 |-- component_12: float (nullable = true)
 |-- component_13: float (nullable = true)
 |-- component_14: float (nullable = true)
 |-- component_15: float (nullable = true)
 |-- component_16: float (nullable = true)
 |-- component_17: float (nullable = true)
 |-- component_18: float (nullable = true)
 |-- component_19: float (nullable = true)
 |-- component_20: floa

In [53]:
# Colonne à enregistrer : 
selected_columns = ["path", "label"] + [f"component_{i+1}" for i in range(num_components)]
df_pca = df_pca.select(*selected_columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_pca.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- component_1: float (nullable = true)
 |-- component_2: float (nullable = true)
 |-- component_3: float (nullable = true)
 |-- component_4: float (nullable = true)
 |-- component_5: float (nullable = true)
 |-- component_6: float (nullable = true)
 |-- component_7: float (nullable = true)
 |-- component_8: float (nullable = true)
 |-- component_9: float (nullable = true)
 |-- component_10: float (nullable = true)
 |-- component_11: float (nullable = true)
 |-- component_12: float (nullable = true)
 |-- component_13: float (nullable = true)
 |-- component_14: float (nullable = true)
 |-- component_15: float (nullable = true)
 |-- component_16: float (nullable = true)
 |-- component_17: float (nullable = true)
 |-- component_18: float (nullable = true)
 |-- component_19: float (nullable = true)
 |-- component_20: float (nullable = true)
 |-- component_21: float (nullable = true)
 |-- component_22: float (nu

In [62]:
# Méthode 01
output_path = PATH_Result_csv + "/pca_csv_1/"
df_pca.write.csv(output_path, header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# Méthode 02
output_path = PATH_Result_csv + "/pca_csv_2/"
df_pca.write.mode("overwrite").option("header","true").csv(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# Méthode 03 : pour enregistrer les parquets dans un seul ficheir csv
output_path = PATH_Result_csv + "/pca_csv_3/"
df_pca.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
# Importation 03 
output_path = PATH_Result_csv + "/pca_csv_3"
Test_pca_3 = pd.read_csv(output_path + "/" + "part-00000-4e64b51d-d7fc-4d4a-aa36-a907ccbefd78-c000.csv")
Test_pca_3.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3110, 765)

In [67]:
type(Test_pca_3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [68]:
Test_pca_3.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['path', 'label', 'component_1', 'component_2', 'component_3',
       'component_4', 'component_5', 'component_6', 'component_7',
       'component_8',
       ...
       'component_754', 'component_755', 'component_756', 'component_757',
       'component_758', 'component_759', 'component_760', 'component_761',
       'component_762', 'component_763'],
      dtype='object', length=765)

# Avec Sample

In [88]:
images_sample = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data_sample)

images_sample.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://projetfruits...|2023-06-21 06:46:42| 57043|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-21 06:46:43| 56929|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-21 06:46:41| 56653|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-21 06:46:43| 56533|[FF D8 FF E0 00 1...|
|s3://projetfruits...|2023-06-21 06:46:44| 56483|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [89]:
images_sample = images_sample.withColumn('label', element_at(split(images_sample['path'], '/'),-2))
print(images_sample.printSchema())
print(images_sample.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-----------------------------------------------------+------+
|path                                                 |label |
+-----------------------------------------------------+------+
|s3://projetfruits/data/input/sample/pear_1/r1_287.jpg|pear_1|
|s3://projetfruits/data/input/sample/pear_1/r1_291.jpg|pear_1|
|s3://projetfruits/data/input/sample/pear_1/r1_283.jpg|pear_1|
|s3://projetfruits/data/input/sample/pear_1/r1_295.jpg|pear_1|
|s3://projetfruits/data/input/sample/pear_1/r1_299.jpg|pear_1|
+-----------------------------------------------------+------+
only showing top 5 rows

None

In [90]:
features_df_sample = images_sample.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
features_df_sample.write.mode("overwrite").parquet(PATH_Result_sample)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
df_for_pca_sample = spark.read.parquet(PATH_Result_sample)
df_for_pca_sample.show(6)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3://projetfruits...|    pear_1|[0.7471891, 0.038...|
|s3://projetfruits...|    pear_1|[1.5626552, 0.022...|
|s3://projetfruits...|    pear_1|[1.1522623, 0.299...|
|s3://projetfruits...|zucchini_1|[0.022995315, 0.5...|
|s3://projetfruits...|zucchini_1|[0.1714293, 1.063...|
|s3://projetfruits...|zucchini_1|[0.0, 0.37462327,...|
+--------------------+----------+--------------------+
only showing top 6 rows

In [93]:
# conversion du type de la colonne 'features' en vecteur pyspark
df_for_pca_sample = df_for_pca_sample.withColumn('features', array_to_vector('features'))
df_for_pca_sample.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3://projetfruits...|    pear_1|[0.74718910455703...|
|s3://projetfruits...|    pear_1|[1.56265521049499...|
|s3://projetfruits...|    pear_1|[1.15226233005523...|
|s3://projetfruits...|zucchini_1|[0.02299531549215...|
|s3://projetfruits...|zucchini_1|[0.17142930626869...|
|s3://projetfruits...|zucchini_1|[0.0,0.3746232688...|
+--------------------+----------+--------------------+
only showing top 6 rows

In [94]:
scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(df_for_pca_sample)

# when we transform the dataframe, the old
# feature will still remain in it
df_scaled_sample = scaler.transform(df_for_pca_sample)
df_scaled_sample.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+
|                path|     label|            features|      scaledFeatures|
+--------------------+----------+--------------------+--------------------+
|s3://projetfruits...|    pear_1|[0.74718910455703...|[0.36056810090543...|
|s3://projetfruits...|    pear_1|[1.56265521049499...|[1.83158191747154...|
|s3://projetfruits...|    pear_1|[1.15226233005523...|[1.09127696432714...|
|s3://projetfruits...|zucchini_1|[0.02299531549215...|[-0.9458001979467...|
|s3://projetfruits...|zucchini_1|[0.17142930626869...|[-0.6780411216462...|
|s3://projetfruits...|zucchini_1|[0.0,0.3746232688...|[-0.9872812926960...|
+--------------------+----------+--------------------+--------------------+
only showing top 6 rows

In [95]:
# PCA
pca = PCA(k=1280, inputCol="scaledFeatures", outputCol="pca_features")
model_pca = pca.fit(df_scaled_sample)
eigenvalues = model_pca.explainedVariance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
df_pca_sample = model_pca.transform(df_scaled_sample)
df_pca_sample.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+
|                path|     label|            features|      scaledFeatures|        pca_features|
+--------------------+----------+--------------------+--------------------+--------------------+
|s3://projetfruits...|    pear_1|[0.74718910455703...|[0.36056810090543...|[-10.084241386939...|
|s3://projetfruits...|    pear_1|[1.56265521049499...|[1.83158191747154...|[-11.032317496145...|
|s3://projetfruits...|    pear_1|[1.15226233005523...|[1.09127696432714...|[-10.819629034799...|
|s3://projetfruits...|zucchini_1|[0.02299531549215...|[-0.9458001979467...|[26.8320046660860...|
|s3://projetfruits...|zucchini_1|[0.17142930626869...|[-0.6780411216462...|[22.0353701394090...|
|s3://projetfruits...|zucchini_1|[0.0,0.3746232688...|[-0.9872812926960...|[9.67264555564796...|
+--------------------+----------+--------------------+--------------------+--------------------+
only showing top 6 rows

In [97]:
# Variance expliquée
print('Explained Variance Ratio', model_pca.explainedVariance.toArray())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio [1.87880509e-01 1.13753486e-01 8.77691404e-02 ... 1.38146375e-17
 8.22699393e-18 3.05775478e-18]

In [98]:
# Calcul de la variance expliquée cumulée
cumulative_variance = np.cumsum(model_pca.explainedVariance.toArray())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
# Trouvez l'indice où la variance expliquée cumulée atteint 99%
index = np.argmax(cumulative_variance >= 0.99)
print('Le nombre de composante retenue : ', index + 1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Le nombre de composante retenue :  268

In [100]:
# On refait une ACP avec 763 composantes : 
# PCA
pca = PCA(k=268, inputCol="scaledFeatures", outputCol="pca_features")
model_pca = pca.fit(df_scaled_sample)
df_pca_sample = model_pca.transform(df_scaled_sample)
df_pca_sample.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+
|                path|     label|            features|      scaledFeatures|        pca_features|
+--------------------+----------+--------------------+--------------------+--------------------+
|s3://projetfruits...|    pear_1|[0.74718910455703...|[0.36056810090543...|[-10.084241386939...|
|s3://projetfruits...|    pear_1|[1.56265521049499...|[1.83158191747154...|[-11.032317496145...|
|s3://projetfruits...|    pear_1|[1.15226233005523...|[1.09127696432714...|[-10.819629034799...|
|s3://projetfruits...|zucchini_1|[0.02299531549215...|[-0.9458001979467...|[26.8320046660860...|
|s3://projetfruits...|zucchini_1|[0.17142930626869...|[-0.6780411216462...|[22.0353701394090...|
|s3://projetfruits...|zucchini_1|[0.0,0.3746232688...|[-0.9872812926960...|[9.67264555564796...|
+--------------------+----------+--------------------+--------------------+--------------------+
only showing top 6 rows

In [101]:
# On ne garde pas la colonne 'features' et on convertit 'pca_features' en array
df_pca_sample = df_pca_sample.select(['path', 'label', vector_to_array('pca_features').alias('pca_features')])
df_pca_sample.show(6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|        pca_features|
+--------------------+----------+--------------------+
|s3://projetfruits...|    pear_1|[-10.084241386939...|
|s3://projetfruits...|    pear_1|[-11.032317496145...|
|s3://projetfruits...|    pear_1|[-10.819629034799...|
|s3://projetfruits...|zucchini_1|[26.8320046660860...|
|s3://projetfruits...|zucchini_1|[22.0353701394090...|
|s3://projetfruits...|zucchini_1|[9.67264555564796...|
+--------------------+----------+--------------------+
only showing top 6 rows

In [102]:
# Enregistrement des données traitées au format "parquet" :
df_pca_sample.repartition(20).write.mode("overwrite").parquet(PATH_Result_pca_sample)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [103]:

df_pca_sample = spark.read.parquet(PATH_Result_pca_sample)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:

df_pca_sample.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: double (containsNull = true)

In [105]:

df_pca_sample.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------------------+
|                path|      label|        pca_features|
+--------------------+-----------+--------------------+
|s3://projetfruits...|apple_red_2|[-11.041967422729...|
|s3://projetfruits...|     pear_1|[-10.385584383483...|
|s3://projetfruits...|apple_red_2|[-9.5711759891598...|
|s3://projetfruits...|apple_red_2|[-11.009383024035...|
|s3://projetfruits...|     pear_1|[-9.9934970576164...|
|s3://projetfruits...|   carrot_1|[34.2660736479951...|
|s3://projetfruits...|   carrot_1|[24.3720665876875...|
|s3://projetfruits...| zucchini_1|[15.9448694061751...|
|s3://projetfruits...|   carrot_1|[38.9940387470131...|
|s3://projetfruits...| zucchini_1|[10.2010107209458...|
|s3://projetfruits...|apple_red_2|[-9.8251354612196...|
|s3://projetfruits...|apple_red_2|[-6.4624367206662...|
|s3://projetfruits...|apple_red_2|[-4.8121548076409...|
|s3://projetfruits...|     pear_1|[-10.625076778630...|
|s3://projetfruits...|   carrot_1|[38.6135653693

In [106]:
# Nombre de composante principale : 
num_components = len(df_pca_sample.select("pca_features").first()[0]) 
num_components

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

268

In [107]:
# Extraction des composante avec udf function
extract_component = udf(lambda x, i: float(x[i]), FloatType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
extract_component

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function <lambda> at 0x7fc8f5b64440>

In [109]:
for i in range(num_components):
    df_pca_sample = df_pca_sample.withColumn(f"component_{i+1}", extract_component("pca_features", lit(i)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
df_pca_sample.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pca_features: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- component_1: float (nullable = true)
 |-- component_2: float (nullable = true)
 |-- component_3: float (nullable = true)
 |-- component_4: float (nullable = true)
 |-- component_5: float (nullable = true)
 |-- component_6: float (nullable = true)
 |-- component_7: float (nullable = true)
 |-- component_8: float (nullable = true)
 |-- component_9: float (nullable = true)
 |-- component_10: float (nullable = true)
 |-- component_11: float (nullable = true)
 |-- component_12: float (nullable = true)
 |-- component_13: float (nullable = true)
 |-- component_14: float (nullable = true)
 |-- component_15: float (nullable = true)
 |-- component_16: float (nullable = true)
 |-- component_17: float (nullable = true)
 |-- component_18: float (nullable = true)
 |-- component_19: float (nullable = true)
 |-- component_20: floa

In [111]:
# Colonne à enregistrer : 
selected_columns = ["path", "label"] + [f"component_{i+1}" for i in range(num_components)]
df_pca_sample = df_pca_sample.select(*selected_columns)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
df_pca_sample.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- component_1: float (nullable = true)
 |-- component_2: float (nullable = true)
 |-- component_3: float (nullable = true)
 |-- component_4: float (nullable = true)
 |-- component_5: float (nullable = true)
 |-- component_6: float (nullable = true)
 |-- component_7: float (nullable = true)
 |-- component_8: float (nullable = true)
 |-- component_9: float (nullable = true)
 |-- component_10: float (nullable = true)
 |-- component_11: float (nullable = true)
 |-- component_12: float (nullable = true)
 |-- component_13: float (nullable = true)
 |-- component_14: float (nullable = true)
 |-- component_15: float (nullable = true)
 |-- component_16: float (nullable = true)
 |-- component_17: float (nullable = true)
 |-- component_18: float (nullable = true)
 |-- component_19: float (nullable = true)
 |-- component_20: float (nullable = true)
 |-- component_21: float (nullable = true)
 |-- component_22: float (nu

In [116]:
# Méthode 01
output_path = PATH_Result_csv_sample + "/pca_csv_1/"
df_pca.write.csv(output_path, header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
# Méthode 02
output_path = PATH_Result_csv_sample + "/pca_csv_2/"
df_pca.write.mode("overwrite").option("header","true").csv(output_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
# Méthode 03 : pour enregistrer les parquets dans un seul ficheir csv
output_path = PATH_Result_csv_sample + "/pca_csv_3/"
df_pca.coalesce(1).write.csv(output_path, header=True, mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
# Importation 03 
output_path = PATH_Result_csv_sample + "/pca_csv_3"
Test_pca_3_sample = pd.read_csv(output_path + "/" + "part-00000-5e36b5ab-81ae-47fb-90c6-778fd5e2fe89-c000.csv")
Test_pca_3_sample.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(3110, 765)

In [123]:
type(Test_pca_3_sample)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [124]:
Test_pca_3_sample.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['path', 'label', 'component_1', 'component_2', 'component_3',
       'component_4', 'component_5', 'component_6', 'component_7',
       'component_8',
       ...
       'component_754', 'component_755', 'component_756', 'component_757',
       'component_758', 'component_759', 'component_760', 'component_761',
       'component_762', 'component_763'],
      dtype='object', length=765)